# Missing Values
Antes de aplicar algum algoritmo de Machine Learning, é necessário trabalhar com as colunas que possuem valores faltantes. É importante destacar que __é imprescindível o entendimento do negócio e da base de dados__, pois apenas desta forma será possível escolher a melhor opção para trabalhar com os valores faltantes.

Neste notebook são apresentadas 3 ténicas. Existem casos que são aplicados algoritmos de Machine Learning para prever os valores faltantes. 

1. [Exclusão de colunas com valores faltantes.](#approach1)
2. [Inclusão de valores através da técnica Imputation.](#approach2)
3. [Inclusão de valores através da técnica Imputation, com o armazenamento de quais valores foram incluídos.](#approach3)

In [1]:
import pandas as pd

df = pd.read_csv('dados.csv', parse_dates=['Date'])
df.head()

,BuildingArea,YearBuilt,Car,Address,Postcode,Rooms,Price,Date
0,NaN,NaN,1.0,85 Turner St,3067.0,2,1480000.0,2016-03-12
1,79.0,1900.0,0.0,25 Bloomburg St,3067.0,2,1035000.0,2016-04-02
2,150.0,1900.0,0.0,5 Charles St,3067.0,3,1465000.0,2017-04-03
3,NaN,NaN,1.0,40 Federation La,3067.0,3,850000.0,2017-04-03
4,142.0,2014.0,2.0,55a Park St,3067.0,4,1600000.0,2016-04-06


## Seleção de _features_
Para manter a análise simples, trabalharemos apenas com as colunas numéricas.

In [2]:
df = df.select_dtypes(exclude=['object', 'datetime64[ns]'])

In [3]:
df.head()

,BuildingArea,YearBuilt,Car,Postcode,Rooms,Price
0,NaN,NaN,1.0,3067.0,2,1480000.0
1,79.0,1900.0,0.0,3067.0,2,1035000.0
2,150.0,1900.0,0.0,3067.0,3,1465000.0
3,NaN,NaN,1.0,3067.0,3,850000.0
4,142.0,2014.0,2.0,3067.0,4,1600000.0


<a id="approach1"></a>

## Colunas com valores faltantes

In [4]:
# Colunas com valores faltantes
df.isnull().sum()

BuildingArea    6450
YearBuilt       5375
Car               62
Postcode           0
Rooms              0
Price              0
dtype: int64

In [5]:
# Armazena os nomes das colunas que possuem valores faltantes
cols_with_missing = [col for col in df if df[col].isnull().any()]
cols_with_missing

['BuildingArea', 'YearBuilt', 'Car']

## Abordagem #1: Excluir colunas com valores faltantes
Esta técnica deve ser evitada devida à perca de dados. Portanto, certifique-se de conhecer o negócio e a base de dados.

In [6]:
# Remove as colunas faltantes do DataFrame
df1 = df.drop(columns=cols_with_missing)

In [7]:
df1.head()

,Postcode,Rooms,Price
0,3067.0,2,1480000.0
1,3067.0,2,1035000.0
2,3067.0,3,1465000.0
3,3067.0,3,850000.0
4,3067.0,4,1600000.0


In [8]:
# Colunas sem valores faltantes
df1.isnull().sum()

Postcode    0
Rooms       0
Price       0
dtype: int64

<a id="approach2"></a>

## Abordagem #2: Imputation
Em seguida, vamos utilizar a classe SimpleImputer para adicionar os valores faltantes em cada coluna.

Doc: [sklearn.impute.SimpleImputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html)

### Média
Ténica para inserir o valor médio.

In [9]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer()
df_mean = pd.DataFrame(imputer.fit_transform(df))

# No processo de Imputation, os nomes das colunas são removidos, portanto devemos incluí-los novamente
df_mean.columns = df.columns

__Neste caso observe que não faz sentido adicionar a média dos Anos na coluna "YearBuilt" e a média dos Carros na coluna "Car"__

In [10]:
df_mean.head()

,BuildingArea,YearBuilt,Car,Postcode,Rooms,Price
0,151.96765,1964.684217,1.0,3067.0,2.0,1480000.0
1,79.00000,1900.000000,0.0,3067.0,2.0,1035000.0
2,150.00000,1900.000000,0.0,3067.0,3.0,1465000.0
3,151.96765,1964.684217,1.0,3067.0,3.0,850000.0
4,142.00000,2014.000000,2.0,3067.0,4.0,1600000.0


### Moda
Ténica para inserir o valor mais frequente.

In [11]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='most_frequent')
df_frequent = pd.DataFrame(imputer.fit_transform(df))

# No processo de Imputation, os nomes das colunas são removidos, portanto devemos incluí-los novamente
df_frequent.columns = df.columns

In [12]:
df_frequent.head()

,BuildingArea,YearBuilt,Car,Postcode,Rooms,Price
0,120.0,1970.0,1.0,3067.0,2.0,1480000.0
1,79.0,1900.0,0.0,3067.0,2.0,1035000.0
2,150.0,1900.0,0.0,3067.0,3.0,1465000.0
3,120.0,1970.0,1.0,3067.0,3.0,850000.0
4,142.0,2014.0,2.0,3067.0,4.0,1600000.0


### Repetição de Valores
Vamos utilizar a função __fillna()__ para preencher os valores faltantes e utilizar o parâmetro __method__ para definir como os valores serão preenchidos.

* __fillna(method='bfill')__: Utiliza o próximo valor válido para preencher os vazios. _hint: b fill - back_
* __fillna(method='ffill')__: Utiliza o último valor válido para preencher os vazios. _hint: f fill - forward_

Doc: [pandas.DataFrame.fillna](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.fillna.html)

In [13]:
# Cria uma cópia do DataFrame para que o original não seja modificado
df_fillna = df.copy()

In [14]:
# Base de dados original. Compare as colunas BuildingArea e YeaBuilt com a linha seguinte
df[['BuildingArea', 'YearBuilt']].head()

,BuildingArea,YearBuilt
0,NaN,NaN
1,79.0,1900.0
2,150.0,1900.0
3,NaN,NaN
4,142.0,2014.0


In [15]:
# Insere os valores faltantes com o próximo valor válido
for col in cols_with_missing:
    df_fillna[col].fillna(method='bfill', inplace=True)

# Base de dados sem os valores faltantes. Compare as colunas BuildingArea e YeaBuilt com a linha superior
df_fillna[['BuildingArea', 'YearBuilt']].head()

,BuildingArea,YearBuilt
0,79.0,1900.0
1,79.0,1900.0
2,150.0,1900.0
3,142.0,2014.0
4,142.0,2014.0


<a id="approach3"></a>

## Abordagem #3: Extensão de Imputation
Agora iremos inserir os valores faltantes, bem como, armazenaremos quais valores foram inseridos criando novas colunas.

In [16]:
# Cria uma cópia do DataFrame para que o original não seja modificado
df_extension = df.copy()

In [17]:
# Criaremos uma coluna com True ou False para os valores faltantes
for col in cols_with_missing:
    df_extension[col + '_missing'] = df_extension[col].isnull()

In [18]:
# Imputation
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='most_frequent')
df_extension_imputed = pd.DataFrame(imputer.fit_transform(df_extension))

# No processo de Imputation, os nomes das colunas são removidos, portanto devemos incluí-los novamente
df_extension_imputed.columns = df_extension.columns

In [19]:
df_extension_imputed.head()

,BuildingArea,YearBuilt,Car,Postcode,Rooms,Price,BuildingArea_missing,YearBuilt_missing,Car_missing
0,120,1970,1,3067,2,1.48e+06,True,True,False
1,79,1900,0,3067,2,1.035e+06,False,False,False
2,150,1900,0,3067,3,1.465e+06,False,False,False
3,120,1970,1,3067,3,850000,True,True,False
4,142,2014,2,3067,4,1.6e+06,False,False,False
